# Crawl Data from Top Youtuber in Taiwan

##  1. Get Famous Youtuber Link

In [1]:
'''
###########################################################
    Import Libraries
###########################################################
'''
import numpy as np
import requests
import sys
import math
import time
from bs4 import BeautifulSoup
import functools

### Get the top 250 youtubers link and basic attributes in taiwan. Sorted by subscribers num

In [2]:
'''
######################################################################################################
    Get the top 250 youtubers link in socialblade website
######################################################################################################
'''

top_youtuber_rank_url = "https://socialblade.com/youtube/top/country/tw/mostsubscribed"

content = requests.get(top_youtuber_rank_url).text
page_soup = BeautifulSoup(content, "html.parser")

# Find potential user position 
div_class = page_soup.find_all('a', href=True)

youtuber_socialblade = [] # Name, url in socialblade
for num in range(0, len(div_class)):
    href = div_class[num]
    url = href.get('href')
    if url.find("/youtube/user/") is not -1:
        youtuber_socialblade.append([href.text, "https://socialblade.com"+url])

print("Top 250 Youtuber: ")
for i in range(0, len(youtuber_socialblade)):
    print(i+1, 'is', youtuber_socialblade[i][0], youtuber_socialblade[i][1])

Top 250 Youtuber: 
1 is 這群人TGOP https://socialblade.com/youtube/user/e84768131
2 is 滾石唱片 ROCK RECORDS https://socialblade.com/youtube/user/rockrecordstaipei
3 is 相信音樂BinMusic https://socialblade.com/youtube/user/binmusictaipei
4 is 阿滴英文 https://socialblade.com/youtube/user/eo3jwe3hezuwfdvcehqk9q
5 is 蔡阿嘎 https://socialblade.com/youtube/user/kyoko38
6 is 阿神 https://socialblade.com/youtube/user/charlie615119
7 is 杰威爾音樂 JVR Music https://socialblade.com/youtube/user/jvrmuzic
8 is 華納音樂 Warner Music Taiwan https://socialblade.com/youtube/user/warnertaiwan
9 is 聖結石Saint https://socialblade.com/youtube/user/idhd_1spj49unbwx1fjs2a
10 is 福茂唱片 https://socialblade.com/youtube/user/linfairrecords
11 is 華研國際 https://socialblade.com/youtube/user/himservice
12 is 眾量級CROWD https://socialblade.com/youtube/user/3lbfxbwtebdooub8-qjm9q
13 is 放火 Louis https://socialblade.com/youtube/user/louislee0602
14 is 瑪莎與熊 Masha and The Bear CH https://socialblade.com/youtube/user/fsxflbi8fpnavq_rl3i4bw
15 is 小玉 https

In [3]:
'''
######################################################################################################
    Get youtuber name, url, UPLOADS, SUBSCRIBERS, VIDEO VIEWS, Country, CHANNEL TYPE, USER CREATED DATE
######################################################################################################
'''
youtuber = []
for i in range(0, len(youtuber_socialblade)): 
    user = youtuber_socialblade[i]
    url = user[1]
    content = requests.get(url).text
    page_soup = BeautifulSoup(content, "html.parser")
    attributes = []
    attributes.append(user[0])
    
    # Get youtube url
    youtube_link_class = page_soup.find_all("a", {"class": "core-button -margin core-small-wide ui-black"})
    for link_text in youtube_link_class:
        link_url = link_text.get('href')
        if link_url is not None:
            attributes.append(link_url)
    
    # Get attributes value
    info_class = page_soup.find_all("div", {"class": "YouTubeUserTopInfo"})
    for info in info_class:
        value = info.find_all("span", {"style": "font-weight: bold;"})[0].text
        attributes.append(value)
    youtuber.append(attributes)
    time.sleep(0.05)
print("Done")

Done


In [4]:
print(type(youtuber))
for i in range(0, len(youtuber)):
    for j in youtuber[i]:
        print(j, end=" "),
    print()

<class 'list'>
這群人TGOP https://youtube.com/channel/UC6FcYHEm7SO1jpu5TKjNXEA 175 2,775,782 588,579,743 TW Entertainment Jun 7th, 2008 
滾石唱片 ROCK RECORDS https://youtube.com/channel/UCPcF3KTqhD67ADkukx_OeDg 4,171 2,215,803 2,504,977,168 TW Music Jun 8th, 2010 
相信音樂BinMusic https://youtube.com/channel/UC7ovu6a8ydIbDy0fAKmoZ9A 1,192 2,109,340 2,279,497,272 TW Music Aug 10th, 2009 
阿滴英文 https://youtube.com/channel/UCeo3JwE3HezUWFdVcehQk9Q 345 1,918,265 138,990,958 TW Education Jan 11th, 2015 
蔡阿嘎 https://youtube.com/channel/UCPwxSX0DYDMQxCvgfeVDv_g 543 1,915,880 418,107,688 TW Entertainment Jun 3rd, 2006 
阿神 https://youtube.com/channel/UCnJEWsS5agXCkqIpyHC9Grg 3,488 1,714,326 638,736,007 TW Games May 11th, 2009 
杰威爾音樂 JVR Music https://youtube.com/channel/UCKUlsqazP-4QmxdEtUPlxOA 516 1,443,296 2,003,057,608 TW Music Sep 2nd, 2010 
華納音樂 Warner Music Taiwan https://youtube.com/channel/UCStpPtoeQUYII1Jsbl0qf8A 1,860 1,413,185 944,679,811 TW Music Jul 6th, 2009 
聖結石Saint https://youtube.com/cha

In [7]:
'''
######################################################################################################
    Preprocess data about comma and date
######################################################################################################
'''
month_dictionary = {'Jan':'01', 'Feb':'02', "Mar":'03', "Apr":'04', "May":'05', "Jun":'06', 
                    "Jul":'07', "Aug":'08', "Sep":'09', "Oct":'10', "Nov":'11', "Dec":'12'}
date_dictionary = {'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07', '8':'08', '9':'09'}

def remove_comma(value_string):
    if type(value_string) == int:
        return value_string
    value = 0
    for i in range(0, len(value_string)):
        if(value_string[i]!=","):
            value = value*10 + int(value_string[i])
    return int(value)
       
def date_to_int(value_string):
    if type(value_string) == int:
        return value_string
    value_string = value_string.replace(',', " ")
    value_string = value_string.replace('th', "")
    value_string = value_string.replace('rd', "")
    value_string = value_string.replace('nd', "")
    value_string = value_string.replace('st', "")
    
    date = value_string.split()
    date[0] = month_dictionary[date[0]]
    if(len(date[1]) == 1):
        date[1] = date_dictionary[date[1]]
    
    date = date[2] + date[0] + date[1]
    
    return int(date)
    
for i in range(0, len(youtuber)):
    youtuber[i][0] = youtuber[i][0].replace(';', "")
    youtuber[i][2] = remove_comma(youtuber[i][2]) #UPLOADS
    youtuber[i][3] = remove_comma(youtuber[i][3]) #SUBSCRIBERS
    youtuber[i][4] = remove_comma(youtuber[i][4]) #VIDEO VIEWS
    youtuber[i][7] = date_to_int(youtuber[i][7]) #Date from string to int
    
    
for i in range(0, len(youtuber)):
    for j in youtuber[i]:
        print(j, end=" "),
    print()

這群人TGOP https://youtube.com/channel/UC6FcYHEm7SO1jpu5TKjNXEA 175 2775782 588579743 TW Entertainment 20080607 
滾石唱片 ROCK RECORDS https://youtube.com/channel/UCPcF3KTqhD67ADkukx_OeDg 4171 2215803 2504977168 TW Music 20100608 
相信音樂BinMusic https://youtube.com/channel/UC7ovu6a8ydIbDy0fAKmoZ9A 1192 2109340 2279497272 TW Music 20090810 
阿滴英文 https://youtube.com/channel/UCeo3JwE3HezUWFdVcehQk9Q 345 1918265 138990958 TW Education 20150111 
蔡阿嘎 https://youtube.com/channel/UCPwxSX0DYDMQxCvgfeVDv_g 543 1915880 418107688 TW Entertainment 20060603 
阿神 https://youtube.com/channel/UCnJEWsS5agXCkqIpyHC9Grg 3488 1714326 638736007 TW Games 20090511 
杰威爾音樂 JVR Music https://youtube.com/channel/UCKUlsqazP-4QmxdEtUPlxOA 516 1443296 2003057608 TW Music 20100902 
華納音樂 Warner Music Taiwan https://youtube.com/channel/UCStpPtoeQUYII1Jsbl0qf8A 1860 1413185 944679811 TW Music 20090706 
聖結石Saint https://youtube.com/channel/UCIdhd_1spj49unBWx1fjS2A 447 1411346 398647979 TW Comedy 20130610 
福茂唱片 https://youtube.com/

## 2. Write basic information of youtuber

In [9]:
'''
######################################################################################################
    Write the basic information of youtuber to csv file
######################################################################################################
'''
import csv
import sys

with open('youtuber.csv', 'w+', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['User_Name', 'Link', 'Uploads', 'Subscribers', 'Video_Views', 'Country', 'Channel_Type', 'User_Created_Date'])
    for i in range(len(youtuber)):
        decode = []
        for info in youtuber[i]:
            if type(info) == str:
                decode.append(info.encode(sys.stdin.encoding, "replace").decode(sys.stdin.encoding))
            else:
                decode.append(info)
        #print(decode)
        writer.writerow(decode)
print("Done")

Done


## 3. Get all videos link in the channel